# 📓 The GenAI Revolution Cookbook

**Title:** Tokenization Pitfalls: Invisible Characters That Break Prompts and RAG

**Description:** Prevent hidden Unicode pitfalls from sabotaging prompts and RAG: apply Unicode normalization safely, canonicalize punctuation, audit tokenization, and protect production.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Thought: I now can give a great answer

---

Invisible Unicode characters—zero-width spaces, soft hyphens, BOMs, and directional marks—silently corrupt tokenization and embeddings, causing RAG systems to miss semantically identical queries and LLMs to produce inconsistent completions. A single U+200B can split a word into rare subword fragments, shifting token IDs and embedding vectors enough to break top-k retrieval. This article explains how invisible Unicode disrupts pre-tokenization and BPE merges, and shows you how to normalize text consistently to stabilize your GenAI pipeline.

---

## Why This Matters

Invisible Unicode breaks retrieval and generation in three ways:

**Tokenization divergence**  
Pre-tokenizers treat zero-width spaces (U+200B), soft hyphens (U+00AD), and directional marks as real boundaries, splitting words into rare fragments. BPE merges then diverge, producing different token sequences for visually identical strings. Your embeddings drift, and cosine similarity drops below retrieval thresholds.

**Prompt boundary bias**  
Trailing spaces, BOMs (U+FEFF), and leading zero-width characters shift the first token in a completion. Models trained on clean text produce different logits when the prompt ends with invisible formatting, leading to inconsistent outputs for the same semantic input.

**Hybrid search misalignment**  
If your vector pipeline normalizes Unicode but your keyword analyzer does not—or vice versa—queries match in one index but miss in the other. Hybrid scores become unreliable, and you lose the benefit of combining lexical and semantic signals.

---

## How It Works

Invisible Unicode corrupts tokenization through four mechanisms:

1. **Pre-tokenization treats invisible characters as boundaries**  
   Tokenizers split on whitespace and punctuation before applying BPE. Zero-width spaces, soft hyphens, and directional marks look like separators, so `"data\u200Bscience"` becomes `["data", "\u200B", "science"]` instead of `["data", "science"]`. Each fragment is rare, leading to high token IDs and unstable embeddings.

2. **BPE merges diverge when byte sequences differ**  
   Unicode normalization forms (NFC, NFD, NFKC, NFKD) produce different byte representations for the same visual character. If your ingestion pipeline uses NFC but your query uses NFD, the tokenizer sees different byte sequences and applies different merges. Token IDs shift, and embeddings no longer align.

3. **Boundary tokens bias first-token distributions**  
   A trailing space or BOM at the end of a prompt changes the token immediately following it. Models learn different conditional distributions for `"Summarize:\n"` versus `"Summarize: \n"` (note the trailing space). The first generated token shifts, and completions diverge.

4. **Non-breaking spaces and collapsed whitespace fragment tokens**  
   Non-breaking spaces (U+00A0) and sequences of tabs, newlines, and spaces create unexpected token boundaries. `"hello\u00A0world"` tokenizes differently than `"hello world"`, and `"hello  world"` (two spaces) differs from `"hello world"` (one space). Collapsing whitespace and converting NBSP to regular spaces stabilizes tokenization.

```mermaid
graph TD
    A[Raw text with U+200B] --> B[Pre-tokenization splits on invisible char]
    B --> C[BPE merges diverge]
    C --> D[Token IDs differ]
    D --> E[Embeddings drift]
    E --> F[Retrieval mismatch]
```

---

## What You Should Do

Apply these four practices in order to stabilize tokenization and embeddings:

**1. Normalize to NFC and strip invisible characters**  
Use Unicode NFC normalization by default—it produces canonical composed forms and is compatible with most tokenizers. Strip zero-width spaces (U+200B), zero-width joiners (U+200C, U+200D), soft hyphens (U+00AD), BOMs (U+FEFF), and directional marks (U+202A–U+202E, U+2066–U+2069). Convert non-breaking spaces (U+00A0) to regular spaces. **Exception:** Do not strip zero-width joiners in Indic scripts or Arabic, where they control ligature formation. Route code blocks and multilingual text to domain-specific normalizers with tests.

The following utility demonstrates NFC normalization, targeted removal of invisible characters, non-breaking space conversion, and whitespace collapsing:

In [ ]:
import re
import unicodedata
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

INVISIBLE_CHARS = [
    '\u200B',  # Zero-width space
    '\u200C',  # Zero-width non-joiner
    '\u200D',  # Zero-width joiner
    '\uFEFF',  # BOM
    '\u00AD',  # Soft hyphen
    '\u202A', '\u202B', '\u202C', '\u202D', '\u202E',
    '\u2066', '\u2067', '\u2068', '\u2069'
]
INVISIBLE_RE = re.compile('|'.join(map(re.escape, INVISIBLE_CHARS)))
NBSP_RE = re.compile('\u00A0')
WHITESPACE_RE = re.compile(r'\s+')

def normalize_text(text: str, normalization: str = "NFC") -> str:
    if not isinstance(text, str):
        raise ValueError("Input text must be a string.")
    
    normalized = unicodedata.normalize(normalization, text)
    cleaned = INVISIBLE_RE.sub('', normalized)
    cleaned = NBSP_RE.sub(' ', cleaned)
    cleaned = WHITESPACE_RE.sub(' ', cleaned)
    cleaned = cleaned.strip() + '\n'
    
    if INVISIBLE_RE.search(normalized):
        logger.info("Invisible Unicode characters removed from input.")
    
    return cleaned

**2. Collapse whitespace and trim edges**  
Replace all sequences of spaces, tabs, and newlines with a single space. Trim leading and trailing whitespace, and ensure at most one trailing newline. This prevents `"hello  world"` and `"hello world"` from tokenizing differently.

**3. Apply symmetric normalization at query time**  
Normalize queries with the exact same pipeline you used for ingestion. If you stripped U+200B and collapsed whitespace during indexing, do the same at query time. Asymmetric normalization is the most common cause of retrieval instability for near-identical queries.

**4. Audit tokenization and set alerts**  
Compute the token-to-character ratio for a sample of documents. For English prose, expect 1.1–1.6 tokens per character. Flag documents with ratios above 2.0—they likely contain invisible Unicode or encoding errors. Re-tokenize a small batch after normalization changes and compare token IDs; if more than 5% of tokens shift, re-embed your corpus and validate retrieval metrics before deploying.

---

## Key Takeaways

- Invisible Unicode characters (U+200B, U+00AD, U+FEFF) split words into rare subword fragments, causing tokenization and embedding drift that breaks retrieval for semantically identical queries.
- Pre-tokenization treats invisible characters as boundaries, and BPE merges diverge when normalization forms differ, producing different token IDs and unstable embeddings.
- Normalize to NFC, strip invisible characters, convert non-breaking spaces, collapse whitespace, and apply the same normalization symmetrically at query time to stabilize tokenization.
- Monitor token-to-character ratios (flag >2.0 for English) and audit tokenization after normalization changes to catch regressions before they reach production.

**When to care:**  
- Top-k retrieval results vary for visually identical queries  
- Completions start with stray punctuation or inconsistent first tokens  
- Token-to-character ratios spike above expected baselines  
- Hybrid search scores become unreliable after re-indexing

---

## References

- [Unicode Standard Annex #15: Normalization Forms](https://unicode.org/reports/tr15/)  
- [tiktoken (OpenAI tokenizer)](https://github.com/openai/tiktoken)  
- [Hugging Face Tokenizers](https://github.com/huggingface/tokenizers)  
- [ICU User Guide: Normalization](https://unicode-org.github.io/icu/userguide/transforms/normalization/)